In [1]:
import os
import pandas as pd
import pathway as pw
import nltk
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

In [2]:
pw.set_license_key('F0EEC7-FA3FF6-216017-196BDE-2DAC9A-V3')
os.environ["GROQ_API_KEY"] = "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"

import logging
logging.basicConfig(level=logging.CRITICAL)

In [3]:
# # Run Once
# import nltk
# nltk.download('punkt', download_dir='/root/nltk_data')
# nltk.download('punkt_tab')

os.environ["RUST_BACKTRACE"] = "full"

In [4]:
""" Get Docs, Parser, Splitter"""

folder = pw.io.fs.read(
    path="./data/",
    format="binary",
    with_metadata=True,
)

sources = [
    folder,
]


from pathway.xpacks.llm import llms, parsers, prompts
# chat = llms.OpenAIChat(model="gpt-4o")
# table_args = {
#    "parsing_algorithm": "llm",
#    "llm": chat,
#    "prompt": prompts.DEFAULT_MD_TABLE_PARSE_PROMPT,
# }
# image_args = {
#     "parsing_algorithm": "llm",
#     "llm": chat,
#     "prompt": prompts.DEFAULT_IMAGE_PARSE_PROMPT,
# }
# parser = parsers.OpenParse(
#     table_args=table_args, 
#     image_args=image_args,
#     processing_pipeline=None # defaults to CustomIngestionPipeline() defined in _openparse_utils, we can change this by defining our own pipeline from openparse
# )
parser = parsers.ParseUnstructured()


from pathway.xpacks.llm.splitters import TokenCountSplitter
# splitter = None # OpenParse handles the text splitting
splitter = TokenCountSplitter()



from pathway.stdlib.indexing.data_index import DataIndex, InnerIndex
from pathway.stdlib.indexing.retrievers import AbstractRetrieverFactory,InnerIndexFactory
from pathway.stdlib.indexing.vector_document_index import default_usearch_knn_document_index
from pathway.xpacks.llm._utils import _coerce_sync

from pathway.xpacks.llm import embedders
embedder = embedders.SentenceTransformerEmbedder(
    model="intfloat/e5-large-v2",
    call_kwargs={}, # optional additional parameters to give to embedder
    device="cpu",
)
#embedder = embedders.OpenAIEmbedder(cache_strategy=DiskCache())

class CustomRetrieverFactor(InnerIndexFactory):
    def build_inner_index():

        return
    def build_index(
        self,
        data_column: pw.ColumnReference,
        data_table: pw.Table,
        metadata_column: pw.ColumnExpression | None = None,
    ) -> DataIndex:
        
        self.embedder = embedder
        self.embedding_dimension = len(_coerce_sync(self.embedder.__wrapped__)("."))
        knn_index = default_usearch_knn_document_index(
            data_column, #chunked_docs.text,
            data_table, #chunked_docs,
            dimensions=self.embedding_dimension,
            metadata_column=metadata_column, #metadata_column=chunked_docs.data["metadata"],
            embedder=self.embedder,
        )
        # inner_index = build_inner_index()
        # knn_index.inner_index 
        return knn_index

retriever_factory = CustomRetrieverFactor()

from pathway.xpacks.llm.document_store import DocumentStore
document_store = DocumentStore(
    *sources,
    retriever_factory=retriever_factory, 
    parser=parser, 
    splitter=splitter,
)

/code/Pathway_RAG/env1/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
# """ IO Test """
# !printf "id,owner,pet\\n1,Alice,dog\\n2,Bob,dog\\n3,Alice,cat\\n4,Bob,dog" > dataset.csv

# import pathway as pw
# class InputSchema(pw.Schema):
#   owner: str
#   pet: str
# t = pw.io.fs.read("dataset.csv", format="csv", schema=InputSchema)
# pw.debug.compute_and_print(t, include_id=False)

In [6]:
""" Make Document Store Servers """

RUST_BACKTRACE=1
host = "127.0.0.1" # define server address
port = 8667 # define server port

from pathway.xpacks.llm.servers import DocumentStoreServer
server = DocumentStoreServer(host=host,port=port,document_store=document_store) # initialize document store server

server.run(threaded=True, with_cache=True, cache_backend=pw.persistence.Backend.filesystem("./Cache")) # start server

/code/Pathway_RAG/env1/lib/python3.12/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Sequence[str] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


<Thread(Thread-6 (run), started 139988129117888)>

In [7]:
""" Company Name from Query - Hardcoded """

import re
def extract_companies_from_question(question):
    pattern = r'between\s+(.+?)\s+and\s+(.+?)\?'
    
    match = re.search(pattern, question)
    
    if match:
        # Extract the two companies
        company_1 = match.group(1).strip()
        company_2 = match.group(2).strip()
        return [company_1, company_2]
    else:
        return None
    
def find_contract_filename(companies, company_contract_df):
    if len(companies) != 2:
        return "Error: Two companies are needed."

    # Normalize company names for case-insensitive matching (strip spaces and lower case)
    company1, company2 = [company.strip().lower() for company in companies]
    # print(f"Companies are:  {company1}, {company2}")

    # Loop through each contract to check if the companies are involved
    for _, row in company_contract_df.iterrows():
        # Clean and normalize the party list in the row
        cleaned_parties = eval(row['Cleaned_Parties'])  # Convert string list to actual list
        cleaned_parties = [party.strip().lower() for party in cleaned_parties]

        # Check if both companies are in the list of parties
        if company1 in cleaned_parties and company2 in cleaned_parties:
            return row['Filename']  # Return the corresponding contract filename
    print(f"No contract found between the companies {companies}")
    return 0

# Example CSV file path
csv_file_path = 'metadata_filename_parties.csv'
company_contract_df = pd.read_csv(csv_file_path)

query = "What is the Agreement Date for the contract between Birch First Global Investments Inc. and Mount Kowledge Holdings Inc.?"

companies = extract_companies_from_question(query)
if companies:
    print(f"The two companies are: {companies}")
    contract_filename = find_contract_filename(companies, company_contract_df)
    print(contract_filename.split(".pdf"))
    l = contract_filename.split(".pdf")
    l.append('.txt')
    contract_filename = ''.join(l)
    print(''.join(l))
    print(f"The contract filename is: {contract_filename}")
else:
    print("No companies found in the query.")


The two companies are: ['Birch First Global Investments Inc.', 'Mount Kowledge Holdings Inc.']
['CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement', '']
CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.txt
The contract filename is: CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.txt


In [8]:
# """ Company Name from Query - LLM Inference """

# summary_template = """
# You are smart assistant that helps find company names from keywords in.
# Given document text, organize it in proper sentences.
# DOCUMENTS:
# {context}
# YOUR ORGANIZED DOCUMENTS:"""

# summary_prompt = ChatPromptTemplate.from_template(summary_template) # Create promt template

# # model = ChatOpenAI()
# summary_model = ChatGroq(temperature=0, model_name="llama3-70b-8192")

# chain_summarize = summary_prompt | summary_model
# if SINGLE_QUERY:
#     summarized_context = chain_summarize.invoke({"context" : retrived_context})
#     print(f"This is the final context {summarized_context.content}")

In [12]:
""" Make DocumentStoreClient """

import json
import requests

class DocumentStoreClient:
    """
    A client you can use to query DocumentStoreServer.

    Please provide either the `url`, or `host` and `port`.

    Args:
        host: host on which `DocumentStoreServer </developers/api-docs/pathway-xpacks-llm/documentstore#pathway.xpacks.llm.document_store.DocumentStoreServer>`_ listens
        port: port on which `DocumentStoreServer </developers/api-docs/pathway-xpacks-llm/documentstore#pathway.xpacks.llm.document_store.DocumentStoreServer>`_ listens
        url: url at which `DocumentStoreServer </developers/api-docs/pathway-xpacks-llm/documentstore#pathway.xpacks.llm.document_store.DocumentStoreServer>`_ listens
        timeout: timeout for the post requests in seconds
    """  # noqa

    def __init__(
        self,
        host: str | None = None,
        port: int | None = None,
        url: str | None = None,
        timeout: int | None = 60,
        additional_headers: dict | None = None,
    ):
        err = "Either (`host` and `port`) or `url` must be provided, but not both."
        if url is not None:
            if host or port:
                raise ValueError(err)
            self.url = url
        else:
            if host is None:
                raise ValueError(err)
            port = port or 80
            self.url = f"http://{host}:{port}"

        self.timeout = timeout
        self.additional_headers = additional_headers or {}

    def query(
        self,
        query: str,
        k: int = 3,
        metadata_filter: str | None = None,
        filepath_globpattern: str | None = None,
    ) -> list[dict]:
        """
        Perform a query to the vector store and fetch results.

        Args:
            query:
            k: number of documents to be returned
            metadata_filter: optional string representing the metadata filtering query
                in the JMESPath format. The search will happen only for documents
                satisfying this filtering.
            filepath_globpattern: optional glob pattern specifying which documents
                will be searched for this query.
        """

        data = {"query": query, "k": k}
        # print("lol")
        # print(data)

        query = data['query']
        # print(query)
        companies = extract_companies_from_question(query)
        # companies = True
        # print(companies)

        if companies:
            # Find filepath
            filepath_globpattern = find_contract_filename(companies, company_contract_df)
            
            # If no contract found between the two companies
            if filepath_globpattern[:11]=="No contract": 
                return 0
            
            # Convert pdf to txt 
            if filepath_globpattern[-4:]==".pdf": 
                filepath_globpattern = filepath_globpattern.split(".pdf")[0]#.replace('-', '_').replace('.', '_').replace(' ','_')
            if filepath_globpattern[-4:]==".PDF":
                filepath_globpattern = filepath_globpattern.split(".PDF")[0]
            filepath_globpattern = "**/" + filepath_globpattern + ".txt"
            
            if metadata_filter is not None:
                data["metadata_filter"] = metadata_filter#f"""contains(path, '{metadata_filter}')"""
            if filepath_globpattern is not None:
                data["filepath_globpattern"] = filepath_globpattern
            url = self.url + "/v1/retrieve"
            # print(data)
            response = requests.post(
                url,
                data=json.dumps(data),
                headers=self._get_request_headers(),
                timeout=self.timeout,
            )

            responses = response.json()
            return sorted(responses, key=lambda x: x["dist"])
        
        else:
            print("Weren't able to trace company")

    # Make an alias
    __call__ = query

    def _get_request_headers(self):
        request_headers = {"Content-Type": "application/json"}
        request_headers.update(self.additional_headers)
        return request_headers
    



# Create client instance
client = DocumentStoreClient(
    host=host,
    port=port,
)


In [14]:
""" Retriver Functionality Test """
SINGLE_QUERY = 0 # Change to see working of a single query 
if SINGLE_QUERY:
    
    # Make query and perform retrieval
    queryy = "What is the Agreement Date for the contract between Birch First Global Investments Inc. and Mount Kowledge Holdings Inc.?"

    # retrieved has keys-> 
    # 'dist' - 
    # 'metadata' 'created_at' - time of file creation
    # 'metadata' 'filetype' - filetype
    # 'metadata' 'languade' - eng usually
    # 'metadata' 'links' - 
    # 'metadata' 'modified_at' - time of file modification
    # 'metadata' 'owner' - owner of file
    # 'metadata' 'path' - path of file
    # 'metadata' 'seen_at' - 
    # 'text' - retrieved chunked docs
    retrieved = client.query(queryy)

    # Get retireved text data context
    retrived_context = []
    for retrieved_doc in retrieved:
        retrived_context.append(retrieved_doc["text"])

    print(f"All retrived context: {retrived_context}")



In [15]:
""" Make the Retrieval A Function """

def retrieve_from_query(query: str, k: int = 3) -> list:

    retrieved = client.query(query, k)
    retrived_context = []
    for retrieved_doc in retrieved:
        retrived_context.append(retrieved_doc["text"])

    return retrived_context

In [16]:
""" Summarizing Agent """

summary_template = """
You are smart assistant that helps organize documents in proper english whilst keeping all information given to you.
Given document text, organize it in proper sentences.
DOCUMENTS:
{context}
YOUR ORGANIZED DOCUMENTS:"""

summary_prompt = ChatPromptTemplate.from_template(summary_template) # Create promt template

# model = ChatOpenAI()
summary_model = ChatGroq(temperature=0, model_name="llama3-70b-8192")

chain_summarize = summary_prompt | summary_model
if SINGLE_QUERY:
    summarized_context = chain_summarize.invoke({"context" : retrived_context})
    print(f"This is the final context {summarized_context.content}")

In [17]:
""" Generator """

generator_template = """
You are smart assistant that helps users with their documents on Google Drive and Sharepoint.
Given a context, respond to the user question.
Give short sentence answers.
If there is no answer from context then say There is no mention of this clause
CONTEXT:
{context}
QUESTION: {question}
YOUR ANSWER:"""

generator_prompt = ChatPromptTemplate.from_template(generator_template)

# llm = ChatOpenAI()
generator_model = ChatGroq(temperature=0, model_name="llama3-70b-8192")

chain_generator = generator_prompt | generator_model
if SINGLE_QUERY:
    response = chain_generator.invoke({"context" : summarized_context.content, "question" : queryy})
    print(f"This is the final context: \n {response.content}")

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

""" Accurary Of RAG Loop """

test_data = pd.read_csv("contract_questions_and_answers.csv")

data_dir_files = [f for f in os.listdir('data') if os.path.isfile(os.path.join('data', f))]

semantic_embedder = SentenceTransformer('all-MiniLM-L6-v2')
error_count = 0
total_count = 0
for index in test_data.index:
    if (test_data.Filename[index].split(".pdf")[0] + '.txt') not in data_dir_files and (test_data.Filename[index].split(".PDF")[0] + '.txt') not in data_dir_files:
        # print(test_data.Filename[index].split(".pdf")[0] + '.txt')
        continue
    print(index)
    # Get Question
    question = test_data.Question[index]

    # Perform retireval and summarization
    retrived_context = retrieve_from_query(question, k=4)
    summarized_context = chain_summarize.invoke({"context" : retrived_context})
    
    # Perform generation
    response = chain_generator.invoke({"context" : summarized_context.content, "question" : question})

    # Perform sematic similarity with true answer
    answer = (test_data.Answer[index].split("]")[0]).split("[")[1]
    response_embedding = semantic_embedder.encode(response.content, convert_to_tensor=True)
    answer_embedding = semantic_embedder.encode(answer, convert_to_tensor=True)
    if answer != '':
        similarity = cosine_similarity([response_embedding], [answer_embedding])[0][0]
    else:
        answer = "There is no mention of this clause"
        answer_embedding = semantic_embedder.encode(answer, convert_to_tensor=True)
        similarity = cosine_similarity([response_embedding], [answer_embedding])[0][0]

    if similarity < 0.4:
        error_count += 1
    
    total_count += 1
    print(f"Question : {question} \n Retireved Context : {retrived_context} \n Response : {response.content} \n Answer : {answer} \n Similarity : {similarity} \n ---------------------")

    

0
Question : What is the Document Name for the contract between Mount Kowledge Holdings Inc. and Birch First Global Investments Inc.? 
 Retireved Context : ['Exhibit 10.27\n\nMARKETING AFFILIATE AGREEMENT\n\nBetween:\n\nBirch First Global Investments Inc.\n\nAnd\n\nMount Knowledge Holdings Inc.\n\nDated: May 8, 2014\n\n1\n\nSource: CYBERGY HOLDINGS, INC., 10-Q, 5/20/2014\n\n1.\n\n2.\n\n2.1\n\n2.2\n\nThis Marketing Affiliate Agreement (the "Agreement") is entered into this 8th day of May 2014, by and between BIRCH FIRST GLOBAL INVESTMENTS INC., a corporation incorporated in the U.S. Virgin Islands, with its main place of business located 9100 Havensight, Port of Sale, Ste. 15/16, St. Thomas, VI 0080 (referred to as "Company") and MOUNT KNOWLEDGE HOLDINGS INC. and/or assigns, a corporation incorporated in the State of Nevada, with its main place of business located at 228 Park Avenue S. #56101 New York, NY 10003\xad1502 (referred to as "Marketing Affiliate" or "MA").\n\nWHEREAS, this Agr

KeyboardInterrupt: 

In [56]:
correct_count = total_count - error_count
accuracy = correct_count / total_count
print(f"Accuracy is {accuracy}")

Accuracy is 0.9473684210526315
